In [17]:
# change two place
def get_balance_sheet(stock_id, start_year, end_year):
    import pandas as pd
    import sqlite3
    import time
    import requests
    
    balance_sheet_url = 'https://mops.twse.com.tw/mops/web/ajax_t164sb03'
    income_statement_url = 'https://mops.twse.com.tw/mops/web/ajax_t164sb04'
    cash_flows_url = 'https://mops.twse.com.tw/mops/web/ajax_t164sb05'
    
    url = cash_flows_url # change 1
    
    year = range(start_year, end_year+1)
    season = range(1, 5)
    dfs = []
    for y in year:
        for s in season:
            form_data = {
                'encodeURIComponent': '',
                'step': '1',
                'firstin': '1',
                'off': '1',
                'queryName': 'co_id',
                'inpuType': 'co_id',
                'TYPEK': 'all',
                'isnew': 'false',
                'co_id': str(stock_id),
                'year': str(y),
                'season': str(s),
            }

            res = requests.post(url, data=form_data)
            table = pd.read_html(res.text)[1]
            table = table.iloc[:, 0:2]
            table.columns = table.columns.get_level_values(2)
            table = table.set_index('會計項目')
            table = table[table.duplicated() == False]
            dfs.append(table)
            time.sleep(3)
            
    df_final = pd.DataFrame(columns=['會計項目'])
    for df in dfs:
        df_final = df_final.merge(df, how='outer', left_index=True, right_index=True)

    df_final = df_final[df_final.index.duplicated() == False]
    df_final = df_final.T.reset_index()
    df_final = df_final.rename(columns={'index': 'date', '會計項目': 'index'})
    df_final = df_final.shift(-1)
    last_index = df_final.index.stop - 1
    df_final = df_final.drop(axis=0, index=last_index)
    conn = sqlite3.connect('balance_sheet.db') # change 2
#     df_final.to_sql('_' + str(stock_id), conn, if_exists='replace')
    return df_final

In [18]:
download_table = [1101, 2317, 2330, 2498]
for d in download_table:
    table = get_balance_sheet(d, 108, 109)
# table
# table_2317 = get_balance_sheet(2317, 102, 104)
# table_2317

In [19]:
table.columns

Index(['date', '其他應付款增加（減少）', '其他應收款（增加）減少', '其他流動負債增加（減少）', '其他流動資產（增加）減少',
       '其他營業資產（增加）減少', '其他金融資產增加', '其他金融資產減少', '其他項目', '利息收入', '利息費用',
       '匯率變動對現金及約當現金之影響', '取得不動產、廠房及設備', '取得投資性不動產', '取得採用權益法之投資', '取得無形資產',
       '取得透過其他綜合損益按公允價值衡量之金融資產', '取得透過損益按公允價值衡量之金融資產', '存入保證金增加', '存入保證金減少',
       '存出保證金減少', '存貨（增加）減少', '強制透過損益按公允價值衡量之金融資產（增加）減少', '應付帳款增加（減少）',
       '應收帳款（增加）減少', '投資活動之淨現金流入（流出）', '折舊費用', '採用權益法認列之關聯企業及合資損失（利益）之份額',
       '攤銷費用', '支付之利息', '收取之利息', '收取之股利', '收益費損項目合計', '期初現金及約當現金餘額',
       '期末現金及約當現金餘額', '本期現金及約當現金增加（減少）數', '營業活動之淨現金流入（流出）', '營業活動之現金流量－間接法',
       '營運產生之現金流入（流出）', '發放現金股利', '短期借款增加', '租賃本金償還', '籌資活動之淨現金流入（流出）',
       '繼續營業單位稅前淨利（淨損）', '股份基礎給付酬勞成本', '股利收入', '與營業活動相關之負債之淨變動合計',
       '與營業活動相關之資產之淨變動合計', '與營業活動相關之資產及負債之淨變動合計', '處分不動產、廠房及設備',
       '處分其他資產損失（利益）', '處分及報廢不動產、廠房及設備損失（利益）', '處分無形資產', '處分無形資產損失（利益）',
       '處分透過其他綜合損益按公允價值衡量之金融資產', '調整項目合計', '負債準備增加（減少）', '退還（支付）之所得稅',
       '透過其他綜合損益按公允價值衡量之金融資產減資退回股款', '非控制權益變動', '非金融資產減損損